# [1321. Restaurant Growth](https://leetcode.com/problems/restaurant-growth/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Customer

<pre>+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| customer_id   | int     |
| name          | varchar |
| visited_on    | date    |
| amount        | int     |
+---------------+---------+</pre>
In SQL,(customer_id, visited_on) is the primary key for this table.
This table contains data about customer transactions in a restaurant.
visited_on is the date on which the customer with ID (customer_id) has visited the restaurant.
amount is the total paid by a customer.


You are the restaurant owner and you want to analyze a possible expansion (there will be at least one customer every day).

Compute the moving average of how much the customer paid in a seven days window (i.e., current day + 6 days before). average_amount should be rounded to two decimal places.

Return the result table ordered by visited_on in ascending order.

The result format is in the following example.



Example 1:

Input:
Customer table:
<pre>+-------------+--------------+--------------+-------------+
| customer_id | name         | visited_on   | amount      |
+-------------+--------------+--------------+-------------+
| 1           | Jhon         | 2019-01-01   | 100         |
| 2           | Daniel       | 2019-01-02   | 110         |
| 3           | Jade         | 2019-01-03   | 120         |
| 4           | Khaled       | 2019-01-04   | 130         |
| 5           | Winston      | 2019-01-05   | 110         |
| 6           | Elvis        | 2019-01-06   | 140         |
| 7           | Anna         | 2019-01-07   | 150         |
| 8           | Maria        | 2019-01-08   | 80          |
| 9           | Jaze         | 2019-01-09   | 110         |
| 1           | Jhon         | 2019-01-10   | 130         |
| 3           | Jade         | 2019-01-10   | 150         |
+-------------+--------------+--------------+-------------+</pre>
Output:
<pre>+--------------+--------------+----------------+
| visited_on   | amount       | average_amount |
+--------------+--------------+----------------+
| 2019-01-07   | 860          | 122.86         |
| 2019-01-08   | 840          | 120            |
| 2019-01-09   | 840          | 120            |
| 2019-01-10   | 1000         | 142.86         |
+--------------+--------------+----------------+</pre>
Explanation:
1st moving average from 2019-01-01 to 2019-01-07 has an average_amount of (100 + 110 + 120 + 130 + 110 + 140 + 150)/7 = 122.86
2nd moving average from 2019-01-02 to 2019-01-08 has an average_amount of (110 + 120 + 130 + 110 + 140 + 150 + 80)/7 = 120
3rd moving average from 2019-01-03 to 2019-01-09 has an average_amount of (120 + 130 + 110 + 140 + 150 + 80 + 110)/7 = 120
4th moving average from 2019-01-04 to 2019-01-10 has an average_amount of (130 + 110 + 140 + 150 + 80 + 110 + 130 + 150)/7 = 142.86

In [7]:
import datetime

# pandas schema
import pandas as pd

data = [[1, 'Jhon', '2019-01-01', 100], [2, 'Daniel', '2019-01-02', 110], [3, 'Jade', '2019-01-03', 120],
        [4, 'Khaled', '2019-01-04', 130], [5, 'Winston', '2019-01-05', 110], [6, 'Elvis', '2019-01-06', 140],
        [7, 'Anna', '2019-01-07', 150], [8, 'Maria', '2019-01-08', 80], [9, 'Jaze', '2019-01-09', 110],
        [1, 'Jhon', '2019-01-10', 130], [3, 'Jade', '2019-01-10', 150]]
       # [1, 'Jhon', '2019-01-10', 130], [3, 'Jade', '2019-01-10', 150],[3, 'Jade', '2019-01-12', 150]]
customer = pd.DataFrame(data, columns=['customer_id', 'name', 'visited_on', 'amount']).astype(
    {'customer_id': 'Int64', 'name': 'object', 'visited_on': 'datetime64[ns]', 'amount': 'Int64'})

#to spark dataframe
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

customer_df = spark.createDataFrame(customer)
customer_df.show()

+-----------+-------+-------------------+------+
|customer_id|   name|         visited_on|amount|
+-----------+-------+-------------------+------+
|          1|   Jhon|2019-01-01 00:00:00|   100|
|          2| Daniel|2019-01-02 00:00:00|   110|
|          3|   Jade|2019-01-03 00:00:00|   120|
|          4| Khaled|2019-01-04 00:00:00|   130|
|          5|Winston|2019-01-05 00:00:00|   110|
|          6|  Elvis|2019-01-06 00:00:00|   140|
|          7|   Anna|2019-01-07 00:00:00|   150|
|          8|  Maria|2019-01-08 00:00:00|    80|
|          9|   Jaze|2019-01-09 00:00:00|   110|
|          1|   Jhon|2019-01-10 00:00:00|   130|
|          3|   Jade|2019-01-10 00:00:00|   150|
+-----------+-------+-------------------+------+



In [72]:
from pyspark.sql import Window
# Solving in Spark DataFrame
wind = Window.orderBy('visited_on').rowsBetween(-6,Window.currentRow)

customer_df\
        .groupBy('visited_on').agg(sum('amount').alias('amt'))\
        .withColumn('amount',sum('amt').over(wind))\
        .withColumn('average_amount',round(avg('amt').over(wind),2))\
        .select('visited_on','amount','average_amount')\
        .show()



+-------------------+------+--------------+
|         visited_on|amount|average_amount|
+-------------------+------+--------------+
|2019-01-01 00:00:00|   100|         100.0|
|2019-01-02 00:00:00|   210|         105.0|
|2019-01-03 00:00:00|   330|         110.0|
|2019-01-04 00:00:00|   460|         115.0|
|2019-01-05 00:00:00|   570|         114.0|
|2019-01-06 00:00:00|   710|        118.33|
|2019-01-07 00:00:00|   860|        122.86|
|2019-01-08 00:00:00|   840|         120.0|
|2019-01-09 00:00:00|   840|         120.0|
|2019-01-10 00:00:00|  1000|        142.86|
+-------------------+------+--------------+



In [73]:
# Solving in spark SQL

customer_df.createOrReplaceTempView('customer')

spark.sql('''
with cte as(
    select visited_on,sum(amount) as amount from customer
    group by visited_on)

select visited_on,amount,average_amount
from
(SELECT visited_on,
    ROUND(SUM(amount) OVER(
        ORDER BY visited_on
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ),2) as amount,
    ROUND(AVG(amount) OVER(
        ORDER BY visited_on
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ),2) as average_amount,
    row_number() over (order by visited_on) as rn
FROM cte)
where rn>0
ORDER BY visited_on

''').show()

+-------------------+------+--------------+
|         visited_on|amount|average_amount|
+-------------------+------+--------------+
|2019-01-01 00:00:00|   100|         100.0|
|2019-01-02 00:00:00|   210|         105.0|
|2019-01-03 00:00:00|   330|         110.0|
|2019-01-04 00:00:00|   460|         115.0|
|2019-01-05 00:00:00|   570|         114.0|
|2019-01-06 00:00:00|   710|        118.33|
|2019-01-07 00:00:00|   860|        122.86|
|2019-01-08 00:00:00|   840|         120.0|
|2019-01-09 00:00:00|   840|         120.0|
|2019-01-10 00:00:00|  1000|        142.86|
+-------------------+------+--------------+

